# Идея проекта, выборка и предобработка данных
<hr>

> TODO: ссылки на методологию анализа фейков по гистограммам, распознавание лиц в OpenCV, исходный датасет

# Обработка данных
<hr>

> TODO: объем и баланс, хар-ки данных на каждом этапе

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("../dfdc_dataframes/df_total_0_4_compact.json")

In [4]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / x)

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2334,326,zrxywvpxnh.mp4,True,48400,5.379489,16.325903,18.552592,11.428242,11.115120,15.687931,...,8.593750,8.984375,16.406250,14.453125,0.500000,0.000021,0.020342,0.005322,2.915663,2.000000
53,54,gfkdvulbfo.mp4,True,42436,14.399969,48.624756,37.622520,23.381528,12.253870,10.675512,...,0.000000,7.653061,4.081633,0.510204,0.473654,0.000024,0.023493,0.005101,3.536333,2.111244
1335,539,faldrzrfbs.mp4,True,93636,18.525085,31.570715,30.552124,29.435766,23.947750,19.088916,...,7.024793,8.471074,7.231405,7.231405,0.264855,0.000011,0.021785,0.002380,5.412486,3.775645
3404,786,veadbrywlp.mp4,True,36100,14.072838,24.728494,23.934091,19.711954,15.402837,13.339693,...,0.510204,7.142857,13.775510,11.734694,0.268698,0.000028,0.044878,0.002985,8.204545,3.721649
128,136,ryccazlbju.mp4,True,91204,22.989569,23.599742,20.215739,22.566286,15.706633,18.644852,...,7.851240,12.396694,13.842975,8.884298,0.167756,0.000011,0.027504,0.002051,13.029143,5.961046


In [5]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,99249.486869,46.130277,524.07128,0.177888,0.295762,0.000016,0.024158,0.003271,10.17117,5.36037


In [6]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,8.383215,4.79785


In [7]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_6_hist_noise_3,10.984466,7.918814,0.162176,3.065652
pt_7_hist_noise_3,12.003171,8.958495,0.145250,3.044676
index,541.650505,405.993939,0.143151,135.656566
pt_13_hist_noise_3,14.806879,11.192997,0.138996,3.613883
pt_8_hist_noise_3,11.521842,8.719307,0.138457,2.802536
pt_9_hist_noise_3,10.486441,7.962488,0.136808,2.523953
pt_12_hist_noise_3,14.922132,11.484739,0.130170,3.437393
pt_44_hist_noise_3,20.006986,15.440608,0.128821,4.566379
pt_43_hist_noise_3,19.735779,15.247012,0.128314,4.488767
pt_24_hist_noise_3,17.134228,13.267918,0.127172,3.866310


In [8]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000016,0.000032
overall_face_hist_std_reversed,0.003057,0.003271,0.006328
overall_face_std_reversed,0.022526,0.024158,0.046684
overall_face_hist_noise_0.5,0.200743,0.177888,0.378631
overall_face_hist_noise_3,0.308146,0.295762,0.603908
pt_7_hist_noise_0.5,1.449724,1.474873,2.924598
pt_9_hist_noise_0.5,1.483363,1.511940,2.995303
pt_8_hist_noise_0.5,1.452484,1.614910,3.067395
pt_10_hist_noise_0.5,1.474690,1.745872,3.220563
pt_6_hist_noise_0.5,1.742977,1.761244,3.504221


In [9]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_6_hist_noise_3,3.065652,18.903280,0.162176
pt_7_hist_noise_3,3.044676,20.961666,0.145250
index,135.656566,947.644444,0.143151
pt_13_hist_noise_3,3.613883,25.999876,0.138996
pt_8_hist_noise_3,2.802536,20.241149,0.138457
pt_9_hist_noise_3,2.523953,18.448929,0.136808
pt_12_hist_noise_3,3.437393,26.406871,0.130170
pt_44_hist_noise_3,4.566379,35.447594,0.128821
pt_43_hist_noise_3,4.488767,34.982791,0.128314
pt_24_hist_noise_3,3.866310,30.402147,0.127172


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [10]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

25


['overall_face_std_reversed',
 'pt_9_hist_noise_3',
 'pt_10_hist_noise_0.5',
 'pt_7_std',
 'pt_6_hist_noise_3',
 'pt_7_hist_noise_3',
 'overall_face_hist_std',
 'overall_face_hist_noise_0.5',
 'face_size_px_reversed',
 'pt_12_hist_noise_3',
 'pt_30_hist_noise_0.5',
 'pt_8_hist_noise_3',
 'overall_face_hist_noise_3',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'pt_1_std',
 'overall_face_hist_noise_0.5_reversed',
 'pt_43_std',
 'pt_24_hist_noise_0.5',
 'pt_22_hist_noise_0.5',
 'pt_13_hist_noise_3',
 'pt_37_std',
 'pt_38_std',
 'overall_face_hist_noise_3_reversed',
 'pt_44_hist_noise_3']

## Разбивка данных на train и test сплиты

In [11]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2334,326,True,48400,5.379489,16.325903,18.552592,11.428242,11.115120,15.687931,10.818629,...,8.593750,8.984375,16.406250,14.453125,0.500000,0.000021,0.020342,0.005322,2.915663,2.000000
53,54,True,42436,14.399969,48.624756,37.622520,23.381528,12.253870,10.675512,16.386357,...,0.000000,7.653061,4.081633,0.510204,0.473654,0.000024,0.023493,0.005101,3.536333,2.111244
1335,539,True,93636,18.525085,31.570715,30.552124,29.435766,23.947750,19.088916,13.616803,...,7.024793,8.471074,7.231405,7.231405,0.264855,0.000011,0.021785,0.002380,5.412486,3.775645
3404,786,True,36100,14.072838,24.728494,23.934091,19.711954,15.402837,13.339693,8.605488,...,0.510204,7.142857,13.775510,11.734694,0.268698,0.000028,0.044878,0.002985,8.204545,3.721649
128,136,True,91204,22.989569,23.599742,20.215739,22.566286,15.706633,18.644852,17.543379,...,7.851240,12.396694,13.842975,8.884298,0.167756,0.000011,0.027504,0.002051,13.029143,5.961046


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [13]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,overall_face_std_reversed,pt_9_hist_noise_3,pt_10_hist_noise_0.5,pt_7_std,pt_6_hist_noise_3,pt_7_hist_noise_3,overall_face_hist_std,overall_face_hist_noise_0.5,face_size_px_reversed,pt_12_hist_noise_3,...,pt_1_std,overall_face_hist_noise_0.5_reversed,pt_43_std,pt_24_hist_noise_0.5,pt_22_hist_noise_0.5,pt_13_hist_noise_3,pt_37_std,pt_38_std,overall_face_hist_noise_3_reversed,pt_44_hist_noise_3
2334,0.020342,12.890625,0.781250,12.615422,10.937500,16.406250,187.905821,0.342975,0.000021,5.078125,...,10.112599,2.915663,19.293781,6.640625,1.562500,8.593750,18.744062,5.148236,2.000000,23.437500
53,0.023493,0.510204,0.000000,12.582742,9.183673,14.285714,196.026006,0.282779,0.000024,0.000000,...,9.015459,3.536333,15.766520,0.000000,0.000000,0.000000,22.056459,12.612900,2.111244,8.163265
1335,0.021785,4.958678,2.066116,15.477121,8.264463,7.231405,420.130030,0.184758,0.000011,10.123967,...,28.682189,5.412486,13.578902,4.545455,3.925620,7.024793,20.049857,17.522583,3.775645,11.570248
3404,0.044878,17.857143,0.000000,17.376358,10.204082,21.428571,335.033918,0.121884,0.000028,2.040816,...,15.830389,8.204545,11.172808,0.000000,0.000000,0.510204,14.637174,12.510640,3.721649,10.204082
128,0.027504,4.958678,1.033058,16.904339,7.851240,9.504132,487.484447,0.076751,0.000011,8.677686,...,11.773766,13.029143,21.668705,4.752066,4.132231,7.851240,23.528371,20.604549,5.961046,8.264463


In [14]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [15]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [16]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=42)

In [17]:
pd.DataFrame(X_train).head()

,overall_face_std_reversed,pt_9_hist_noise_3,pt_10_hist_noise_0.5,pt_7_std,pt_6_hist_noise_3,pt_7_hist_noise_3,overall_face_hist_std,overall_face_hist_noise_0.5,face_size_px_reversed,pt_12_hist_noise_3,...,pt_1_std,overall_face_hist_noise_0.5_reversed,pt_43_std,pt_24_hist_noise_0.5,pt_22_hist_noise_0.5,pt_13_hist_noise_3,pt_37_std,pt_38_std,overall_face_hist_noise_3_reversed,pt_44_hist_noise_3
2266,0.014548,3.298611,0.347222,33.047307,13.541667,19.444444,364.042161,0.153369,0.000009,5.902778,...,45.669507,6.520242,14.238176,0.347222,0.694444,7.812500,34.412580,22.659363,4.445620,8.680556
540,0.033799,4.513889,0.694444,8.068379,5.381944,5.034722,943.999692,0.130792,0.000009,3.819444,...,15.783850,7.645755,25.315663,1.562500,6.076389,2.083333,10.823082,25.452521,6.004294,16.319444
3276,0.013091,9.500000,0.250000,5.402925,1.750000,2.500000,721.575081,0.208292,0.000012,18.500000,...,5.088978,4.800952,15.084485,2.500000,0.500000,16.250000,12.029371,10.296210,3.332893,21.750000
2862,0.018456,37.244898,0.000000,25.450245,22.959184,28.061224,113.686418,0.312500,0.000025,26.530612,...,50.048830,3.200000,31.223319,0.000000,0.000000,32.653061,41.755407,31.696179,1.731602,45.918367
1983,0.020853,7.544379,0.000000,23.368135,9.911243,8.284024,1114.859948,0.115194,0.000007,2.218935,...,57.849609,8.681026,38.519004,6.804734,4.585799,5.769231,50.693916,43.093966,7.016788,17.751479


In [18]:
y_train.head()

2266    False
540      True
3276     True
2862     True
1983    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [20]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [21]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [22]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[57 48]
 [32 61]]
               precision    recall  f1-score   support

 class "real"       0.64      0.54      0.59       105
class "fakes"       0.56      0.66      0.60        93

     accuracy                           0.60       198
    macro avg       0.60      0.60      0.60       198
 weighted avg       0.60      0.60      0.60       198



### SVC (f1=0.64) [to be modified]

In [23]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[50 55]
 [16 77]]
               precision    recall  f1-score   support

 class "real"       0.76      0.48      0.58       105
class "fakes"       0.58      0.83      0.68        93

     accuracy                           0.64       198
    macro avg       0.67      0.65      0.63       198
 weighted avg       0.68      0.64      0.63       198



### LogisticRegression (f1=0.61) [to be modified]

In [24]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[56 49]
 [28 65]]
               precision    recall  f1-score   support

 class "real"       0.67      0.53      0.59       105
class "fakes"       0.57      0.70      0.63        93

     accuracy                           0.61       198
    macro avg       0.62      0.62      0.61       198
 weighted avg       0.62      0.61      0.61       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [25]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(30,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[65 40]
 [28 65]]
               precision    recall  f1-score   support

 class "real"       0.70      0.62      0.66       105
class "fakes"       0.62      0.70      0.66        93

     accuracy                           0.66       198
    macro avg       0.66      0.66      0.66       198
 weighted avg       0.66      0.66      0.66       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [26]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=5, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[58 47]
 [24 69]]
               precision    recall  f1-score   support

 class "real"       0.71      0.55      0.62       105
class "fakes"       0.59      0.74      0.66        93

     accuracy                           0.64       198
    macro avg       0.65      0.65      0.64       198
 weighted avg       0.65      0.64      0.64       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [27]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[45 60]
 [35 58]]
               precision    recall  f1-score   support

 class "real"       0.56      0.43      0.49       105
class "fakes"       0.49      0.62      0.55        93

     accuracy                           0.52       198
    macro avg       0.53      0.53      0.52       198
 weighted avg       0.53      0.52      0.52       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [28]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[28 77]
 [11 82]]
               precision    recall  f1-score   support

 class "real"       0.72      0.27      0.39       105
class "fakes"       0.52      0.88      0.65        93

     accuracy                           0.56       198
    macro avg       0.62      0.57      0.52       198
 weighted avg       0.62      0.56      0.51       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [29]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=3, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)

[[61 44]
 [18 75]]
               precision    recall  f1-score   support

 class "real"       0.77      0.58      0.66       105
class "fakes"       0.63      0.81      0.71        93

     accuracy                           0.69       198
    macro avg       0.70      0.69      0.69       198
 weighted avg       0.71      0.69      0.68       198

